# Training and Evaluating Logistic Regression Model on Encrypted data 

in this notebook we are going to use Homomorphic Encryption to encrypt data before giving it to the LR model for evaluation, and for this we are going to use the HR Lib TenSEAL 